# Creation of Reviews CSV File including Bitcoin Conversion
This notebook queries all positive vendor reviews of Berlusconi market from the SQL database and converts the price in euro into Bitcoin in Satoshi. The resulting dataframe is then stored as csv to be easily loadable from other notebooks.

In [2]:
# import needed libraries
import tqdm
import pickle
import pandas as pd
import mysql.connector

In [3]:
# read bitcoin conversion rates file
df_btc = pd.read_csv("BTC_EUR_conversion.csv", delimiter=",")
df_btc.Date = pd.to_datetime(df_btc['Date'])
df_btc.Price = df_btc.Price.str.replace(",", "")
df_btc.Date = df_btc.Date.dt.date
df_btc.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,2019-12-31,6397.4,"6,449.8","6,511.5","6,335.3",3.03K,-0.81%
1,2019-12-30,6449.8,"6,606.7","6,606.7","6,430.1",3.56K,-2.37%
2,2019-12-29,6606.7,"6,536.7","6,729.8","6,519.4",2.11K,1.07%
3,2019-12-28,6536.6,"6,485.0","6,584.8","6,473.2",1.69K,0.80%
4,2019-12-27,6485.0,"6,489.9","6,507.3","6,352.0",2.84K,-0.08%


In [4]:
# connect to sql database
df_pw = pd.read_csv("pw.csv", delimiter=",")
con = mysql.connector.connect(
    host="localhost",
    user=df_pw["name"][0], 
    passwd=df_pw["password"][0],
    database="DarknetScrape",
)

In [5]:
# get all positive reviews for berlusconi market
cursor = con.cursor()
query = """SELECT price_eur,
`time` ,
customer,
vendor_id,
vendor_review_id

FROM DarknetScrape.vendor_reviews
where market = 'Berlusconi Market'
and positive = 1
;
"""
cursor.execute(query)
results = cursor.fetchall()

df = pd.DataFrame(results, columns=["price_eur","time","customer","vendor_id","vendor_review_id"])
df.head()

,price_eur,time,customer,vendor_id,vendor_review_id
0,12.03,2019-07-23 19:30:31,G***2,1e5f1efa34afe2206b99afaa903f8ee8,2
1,12.19,2019-07-23 07:47:21,n***1,1e5f1efa34afe2206b99afaa903f8ee8,3
2,12.22,2019-07-21 19:14:31,n***o,1e5f1efa34afe2206b99afaa903f8ee8,4
3,7.33,2019-07-21 19:13:00,n***o,1e5f1efa34afe2206b99afaa903f8ee8,5
4,13.29,2019-07-19 07:07:59,b***o,1e5f1efa34afe2206b99afaa903f8ee8,6


In [6]:
# convert the euro price from the reviews into bitcoin in satoshi (value at the time)
df["price_btc_review"] = 0.0
df["price_btc_reviews_satoshi"] = 0.0
for i, row in df.iterrows():
    price_btc_review = float(row.price_eur) / float(df_btc[df_btc["Date"] == row.time.date()]["Price"].iloc[0])
    df.at[i,'price_btc_review'] = price_btc_review
    df.at[i,'price_btc_reviews_satoshi'] = price_btc_review * 100000000
df.head()

,price_eur,time,customer,vendor_id,vendor_review_id,price_btc_review,price_btc_reviews_satoshi
0,12.03,2019-07-23 19:30:31,G***2,1e5f1efa34afe2206b99afaa903f8ee8,2,0.001359,135921.452540
1,12.19,2019-07-23 07:47:21,n***1,1e5f1efa34afe2206b99afaa903f8ee8,3,0.001377,137729.219158
2,12.22,2019-07-21 19:14:31,n***o,1e5f1efa34afe2206b99afaa903f8ee8,4,0.001293,129327.223275
3,7.33,2019-07-21 19:13:00,n***o,1e5f1efa34afe2206b99afaa903f8ee8,5,0.000776,77575.167480
4,13.29,2019-07-19 07:07:59,b***o,1e5f1efa34afe2206b99afaa903f8ee8,6,0.001415,141455.211172


In [7]:
# sort by time for more efficient querying in neo4j when looping over reviews
df.time = pd.to_datetime(df.time)
df = df.sort_values(by="time",ascending=True)
df.reset_index(drop=True, inplace=True)

In [8]:
# store results as csv
df.to_csv("reviews_price_btc.csv", index=False)

In [9]:
# check number of reviews
len(df)

31277

In [10]:
display(df[["vendor_review_id","vendor_id","customer","time","price_eur","price_btc_reviews_satoshi"]])

,vendor_review_id,vendor_id,customer,time,price_eur,price_btc_reviews_satoshi
0,20309,0165d25d07762fda181f480bb0f005aa,g***o,2017-08-08 09:57:08,3.07,1.048569e+05
1,3942,107179581d76c12ece2679738f4ca066,T***e,2017-08-09 13:55:34,2.22,7.727921e+04
2,9567,054a13aa37ed397216c11ea660236618,L***x,2017-08-11 13:06:05,69.67,2.234159e+06
3,3941,107179581d76c12ece2679738f4ca066,J***s,2017-08-17 22:58:43,4.20,1.154798e+05
4,3940,107179581d76c12ece2679738f4ca066,t***k,2017-08-18 04:20:59,4.08,1.155807e+05
...,...,...,...,...,...,...
31272,104268,8e0ee15c432e92ca315057dacdf67058,by [5],2019-07-29 15:04:25,29.03,3.406677e+05
31273,27653,a86eaa443894c015eb7ff280d7597ed7,C***t,2019-07-29 15:05:14,41.87,4.913454e+05
31274,27652,a86eaa443894c015eb7ff280d7597ed7,b***6,2019-07-29 15:10:23,60.24,7.069178e+05
31275,26604,7ae5fd6d6e8d45455fd29d0c7fd6b3f7,k***s,2019-07-29 15:11:53,72.25,8.478554e+05
